In [8]:
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow-federated-nightly
!pip install --quiet --upgrade nest-asyncio
!pip install --quiet --upgrade tb-nightly  # or tensorboard, but not both

import nest_asyncio
nest_asyncio.apply()

Found existing installation: tb-nightly 2.7.0a20211013
Uninstalling tb-nightly-2.7.0a20211013:


  Successfully uninstalled tb-nightly-2.7.0a20211013


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\nest_asyncio.py'
Consider using the `--user` option or check the permissions.

ERROR: Invalid requirement: '#'


In [9]:
import collections
import pickle
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import random
np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [10]:
def noisy_client_taken(client_taken,client_percent, data_percent):
    client_data={}
    initial='clients'
    random_client = random.sample(range(0,client_taken), round(client_taken*client_percent))
    random_client = np.sort(random_client)
    print(random_client)
    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
    x=0
    y=0
    for i in range(client_taken):
        client_dataset = emnist_train.create_tf_dataset_for_client(
          emnist_train.client_ids[i])
        client_name=['{}_{}'.format(initial, i+1)]
        bla=np.zeros((1,784))
        sla= np.zeros((1,1))
        if random_client[y]==x:
            print("engaged")
            
            data_lenght=len(client_dataset)
            random_data= random.sample(range(0,data_lenght), round(data_lenght*data_percent))
            random_data= np.sort(random_data)
#             print(random_data)
            n=0
            j=0
            for example in client_dataset:
            # Append counts individually per label to make plots
            # more colorful instead of one color per plot.
                label = example['label'].numpy()
                label= label.reshape(-1,1)
                label= np.array(label)
                sla= np.concatenate((sla,label))
                data = example['pixels'].numpy()
                data = data.reshape(1,784)
                if random_data[j]==n:
                    mu, sigma = 0, 0.7 
                    noise = np.random.normal(mu, sigma, [1,784])
                    data= data + noise
                    data= np.array(data)
                    data= (data - np.min(data)) / (np.max(data) - np.min(data))
                    bla= np.concatenate((bla,data))
                    if j<len(random_data)-1:
                        j= j+1
#                     print(j)
                else:
                    bla= np.concatenate((bla,data))
                
                n+=1
            if y<len(random_client)-1:
                y= y+1
        else:
            print('non_engaed')
            for example in client_dataset:
        # Append counts individually per label to make plots
        # more colorful instead of one color per plot.
                label = example['label'].numpy()
                label= label.reshape(-1,1)
                label= np.array(label)
                sla= np.concatenate((sla,label))
                data = example['pixels'].numpy()
                data = data.reshape(1,784)
                bla= np.concatenate((bla,data))
        x= x+1
        print(x,y)

                

        bla= bla[1:len(bla),:]
        sla= sla[1:len(sla),:]
#         sla= tf.keras.utils.to_categorical(sla, 10)
        dataset= list(zip(bla,sla))
        present_data= {client_name[0]:dataset}
        client_data.update(present_data)
    return client_data, random_client

In [25]:
def test_client_taken(client_taken):
    bla=np.zeros((1,784))
    sla= np.zeros((1,1))
    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
    for i in range(client_taken):
        client_dataset = emnist_test.create_tf_dataset_for_client(
          emnist_test.client_ids[i])
        print('client Id: ',i)
        for example in client_dataset:
        # Append counts individually per label to make plots
        # more colorful instead of one color per plot.
            label = example['label'].numpy()
            label= label.reshape(-1,1)
            label= np.array(label)
            sla= np.concatenate((sla,label))
            data = example['pixels'].numpy()
            data = data.reshape(1,784)
            bla= np.concatenate((bla,data))

    bla= bla[1:len(bla),:]
    sla= sla[1:len(sla),:]
#     sla= tf.keras.utils.to_categorical(sla, 10)
    return bla, sla

In [12]:
def save_file(file_name,data):
    open_file = open(file_name, "wb")
    pickle.dump(data, open_file)
    open_file.close()


In [21]:
#train data creation
num_client=3000
test_client= 3000
client_percent= .1
data_percent= 1
client_data, bad_client= noisy_client_taken(num_client, client_percent,data_percent)


[   9   11   54   56   57   64   67   82   90   93  114  115  122  144
  152  159  186  191  194  202  204  207  224  231  235  238  250  269
  280  283  290  305  315  337  340  385  389  390  391  394  414  426
  436  437  438  440  471  483  490  507  511  518  520  528  530  543
  554  584  604  608  633  649  659  682  683  685  702  713  730  743
  753  759  791  802  820  824  830  836  838  840  841  849  871  879
  883  890  895  897  900  906  911  914  918  922  923  947  952  956
  959  967  972  978  988  997  998 1000 1001 1018 1027 1043 1046 1058
 1064 1080 1087 1110 1126 1131 1135 1154 1156 1162 1181 1196 1198 1200
 1209 1214 1216 1222 1235 1236 1238 1239 1258 1260 1271 1296 1322 1324
 1332 1337 1345 1346 1370 1382 1391 1392 1399 1426 1430 1434 1437 1439
 1443 1453 1454 1455 1499 1504 1509 1550 1552 1558 1559 1585 1588 1636
 1640 1647 1648 1652 1668 1695 1706 1719 1761 1788 1797 1807 1819 1869
 1876 1893 1902 1926 1927 1942 1952 1955 1958 1972 1996 1997 2006 2007
 2017 

392 39
non_engaed
393 39
non_engaed
394 39
engaged
395 40
non_engaed
396 40
non_engaed
397 40
non_engaed
398 40
non_engaed
399 40
non_engaed
400 40
non_engaed
401 40
non_engaed
402 40
non_engaed
403 40
non_engaed
404 40
non_engaed
405 40
non_engaed
406 40
non_engaed
407 40
non_engaed
408 40
non_engaed
409 40
non_engaed
410 40
non_engaed
411 40
non_engaed
412 40
non_engaed
413 40
non_engaed
414 40
engaged
415 41
non_engaed
416 41
non_engaed
417 41
non_engaed
418 41
non_engaed
419 41
non_engaed
420 41
non_engaed
421 41
non_engaed
422 41
non_engaed
423 41
non_engaed
424 41
non_engaed
425 41
non_engaed
426 41
engaged
427 42
non_engaed
428 42
non_engaed
429 42
non_engaed
430 42
non_engaed
431 42
non_engaed
432 42
non_engaed
433 42
non_engaed
434 42
non_engaed
435 42
non_engaed
436 42
engaged
437 43
engaged
438 44
engaged
439 45
non_engaed
440 45
engaged
441 46
non_engaed
442 46
non_engaed
443 46
non_engaed
444 46
non_engaed
445 46
non_engaed
446 46
non_engaed
447 46
non_engaed
448 46
non_en

863 82
non_engaed
864 82
non_engaed
865 82
non_engaed
866 82
non_engaed
867 82
non_engaed
868 82
non_engaed
869 82
non_engaed
870 82
non_engaed
871 82
engaged
872 83
non_engaed
873 83
non_engaed
874 83
non_engaed
875 83
non_engaed
876 83
non_engaed
877 83
non_engaed
878 83
non_engaed
879 83
engaged
880 84
non_engaed
881 84
non_engaed
882 84
non_engaed
883 84
engaged
884 85
non_engaed
885 85
non_engaed
886 85
non_engaed
887 85
non_engaed
888 85
non_engaed
889 85
non_engaed
890 85
engaged
891 86
non_engaed
892 86
non_engaed
893 86
non_engaed
894 86
non_engaed
895 86
engaged
896 87
non_engaed
897 87
engaged
898 88
non_engaed
899 88
non_engaed
900 88
engaged
901 89
non_engaed
902 89
non_engaed
903 89
non_engaed
904 89
non_engaed
905 89
non_engaed
906 89
engaged
907 90
non_engaed
908 90
non_engaed
909 90
non_engaed
910 90
non_engaed
911 90
engaged
912 91
non_engaed
913 91
non_engaed
914 91
engaged
915 92
non_engaed
916 92
non_engaed
917 92
non_engaed
918 92
engaged
919 93
non_engaed
920 93


1294 137
non_engaed
1295 137
non_engaed
1296 137
engaged
1297 138
non_engaed
1298 138
non_engaed
1299 138
non_engaed
1300 138
non_engaed
1301 138
non_engaed
1302 138
non_engaed
1303 138
non_engaed
1304 138
non_engaed
1305 138
non_engaed
1306 138
non_engaed
1307 138
non_engaed
1308 138
non_engaed
1309 138
non_engaed
1310 138
non_engaed
1311 138
non_engaed
1312 138
non_engaed
1313 138
non_engaed
1314 138
non_engaed
1315 138
non_engaed
1316 138
non_engaed
1317 138
non_engaed
1318 138
non_engaed
1319 138
non_engaed
1320 138
non_engaed
1321 138
non_engaed
1322 138
engaged
1323 139
non_engaed
1324 139
engaged
1325 140
non_engaed
1326 140
non_engaed
1327 140
non_engaed
1328 140
non_engaed
1329 140
non_engaed
1330 140
non_engaed
1331 140
non_engaed
1332 140
engaged
1333 141
non_engaed
1334 141
non_engaed
1335 141
non_engaed
1336 141
non_engaed
1337 141
engaged
1338 142
non_engaed
1339 142
non_engaed
1340 142
non_engaed
1341 142
non_engaed
1342 142
non_engaed
1343 142
non_engaed
1344 142
non_en

1710 175
non_engaed
1711 175
non_engaed
1712 175
non_engaed
1713 175
non_engaed
1714 175
non_engaed
1715 175
non_engaed
1716 175
non_engaed
1717 175
non_engaed
1718 175
non_engaed
1719 175
engaged
1720 176
non_engaed
1721 176
non_engaed
1722 176
non_engaed
1723 176
non_engaed
1724 176
non_engaed
1725 176
non_engaed
1726 176
non_engaed
1727 176
non_engaed
1728 176
non_engaed
1729 176
non_engaed
1730 176
non_engaed
1731 176
non_engaed
1732 176
non_engaed
1733 176
non_engaed
1734 176
non_engaed
1735 176
non_engaed
1736 176
non_engaed
1737 176
non_engaed
1738 176
non_engaed
1739 176
non_engaed
1740 176
non_engaed
1741 176
non_engaed
1742 176
non_engaed
1743 176
non_engaed
1744 176
non_engaed
1745 176
non_engaed
1746 176
non_engaed
1747 176
non_engaed
1748 176
non_engaed
1749 176
non_engaed
1750 176
non_engaed
1751 176
non_engaed
1752 176
non_engaed
1753 176
non_engaed
1754 176
non_engaed
1755 176
non_engaed
1756 176
non_engaed
1757 176
non_engaed
1758 176
non_engaed
1759 176
non_engaed
176

2127 212
non_engaed
2128 212
non_engaed
2129 212
non_engaed
2130 212
non_engaed
2131 212
non_engaed
2132 212
non_engaed
2133 212
non_engaed
2134 212
non_engaed
2135 212
non_engaed
2136 212
non_engaed
2137 212
non_engaed
2138 212
non_engaed
2139 212
non_engaed
2140 212
non_engaed
2141 212
non_engaed
2142 212
non_engaed
2143 212
non_engaed
2144 212
non_engaed
2145 212
non_engaed
2146 212
non_engaed
2147 212
non_engaed
2148 212
non_engaed
2149 212
non_engaed
2150 212
non_engaed
2151 212
non_engaed
2152 212
non_engaed
2153 212
engaged
2154 213
non_engaed
2155 213
non_engaed
2156 213
non_engaed
2157 213
engaged
2158 214
non_engaed
2159 214
non_engaed
2160 214
non_engaed
2161 214
non_engaed
2162 214
non_engaed
2163 214
non_engaed
2164 214
non_engaed
2165 214
non_engaed
2166 214
non_engaed
2167 214
engaged
2168 215
non_engaed
2169 215
non_engaed
2170 215
non_engaed
2171 215
non_engaed
2172 215
non_engaed
2173 215
non_engaed
2174 215
non_engaed
2175 215
non_engaed
2176 215
non_engaed
2177 215


2543 244
non_engaed
2544 244
non_engaed
2545 244
non_engaed
2546 244
non_engaed
2547 244
non_engaed
2548 244
non_engaed
2549 244
non_engaed
2550 244
engaged
2551 245
non_engaed
2552 245
non_engaed
2553 245
non_engaed
2554 245
non_engaed
2555 245
non_engaed
2556 245
non_engaed
2557 245
non_engaed
2558 245
non_engaed
2559 245
non_engaed
2560 245
non_engaed
2561 245
non_engaed
2562 245
non_engaed
2563 245
non_engaed
2564 245
non_engaed
2565 245
non_engaed
2566 245
non_engaed
2567 245
non_engaed
2568 245
engaged
2569 246
non_engaed
2570 246
non_engaed
2571 246
engaged
2572 247
non_engaed
2573 247
non_engaed
2574 247
non_engaed
2575 247
non_engaed
2576 247
non_engaed
2577 247
engaged
2578 248
engaged
2579 249
non_engaed
2580 249
non_engaed
2581 249
non_engaed
2582 249
non_engaed
2583 249
non_engaed
2584 249
engaged
2585 250
non_engaed
2586 250
non_engaed
2587 250
non_engaed
2588 250
non_engaed
2589 250
non_engaed
2590 250
non_engaed
2591 250
non_engaed
2592 250
non_engaed
2593 250
non_engae

2966 294
non_engaed
2967 294
non_engaed
2968 294
engaged
2969 295
engaged
2970 296
engaged
2971 297
non_engaed
2972 297
engaged
2973 298
non_engaed
2974 298
non_engaed
2975 298
non_engaed
2976 298
non_engaed
2977 298
non_engaed
2978 298
non_engaed
2979 298
non_engaed
2980 298
non_engaed
2981 298
engaged
2982 299
non_engaed
2983 299
non_engaed
2984 299
non_engaed
2985 299
non_engaed
2986 299
non_engaed
2987 299
non_engaed
2988 299
non_engaed
2989 299
non_engaed
2990 299
non_engaed
2991 299
non_engaed
2992 299
non_engaed
2993 299
non_engaed
2994 299
non_engaed
2995 299
engaged
2996 299
non_engaed
2997 299
non_engaed
2998 299
non_engaed
2999 299
non_engaed
3000 299


In [26]:
# test data creation
test_data, test_label= test_client_taken(num_client)

client Id:  0
client Id:  1
client Id:  2
client Id:  3
client Id:  4
client Id:  5
client Id:  6
client Id:  7
client Id:  8
client Id:  9
client Id:  10
client Id:  11
client Id:  12
client Id:  13
client Id:  14
client Id:  15
client Id:  16
client Id:  17
client Id:  18
client Id:  19
client Id:  20
client Id:  21
client Id:  22
client Id:  23
client Id:  24
client Id:  25
client Id:  26
client Id:  27
client Id:  28
client Id:  29
client Id:  30
client Id:  31
client Id:  32
client Id:  33
client Id:  34
client Id:  35
client Id:  36
client Id:  37
client Id:  38
client Id:  39
client Id:  40
client Id:  41
client Id:  42
client Id:  43
client Id:  44
client Id:  45
client Id:  46
client Id:  47
client Id:  48
client Id:  49
client Id:  50
client Id:  51
client Id:  52
client Id:  53
client Id:  54
client Id:  55
client Id:  56
client Id:  57
client Id:  58
client Id:  59
client Id:  60
client Id:  61
client Id:  62
client Id:  63
client Id:  64
client Id:  65
client Id:  66
clien

client Id:  520
client Id:  521
client Id:  522
client Id:  523
client Id:  524
client Id:  525
client Id:  526
client Id:  527
client Id:  528
client Id:  529
client Id:  530
client Id:  531
client Id:  532
client Id:  533
client Id:  534
client Id:  535
client Id:  536
client Id:  537
client Id:  538
client Id:  539
client Id:  540
client Id:  541
client Id:  542
client Id:  543
client Id:  544
client Id:  545
client Id:  546
client Id:  547
client Id:  548
client Id:  549
client Id:  550
client Id:  551
client Id:  552
client Id:  553
client Id:  554
client Id:  555
client Id:  556
client Id:  557
client Id:  558
client Id:  559
client Id:  560
client Id:  561
client Id:  562
client Id:  563
client Id:  564
client Id:  565
client Id:  566
client Id:  567
client Id:  568
client Id:  569
client Id:  570
client Id:  571
client Id:  572
client Id:  573
client Id:  574
client Id:  575
client Id:  576
client Id:  577
client Id:  578
client Id:  579
client Id:  580
client Id:  581
client I

client Id:  1031
client Id:  1032
client Id:  1033
client Id:  1034
client Id:  1035
client Id:  1036
client Id:  1037
client Id:  1038
client Id:  1039
client Id:  1040
client Id:  1041
client Id:  1042
client Id:  1043
client Id:  1044
client Id:  1045
client Id:  1046
client Id:  1047
client Id:  1048
client Id:  1049
client Id:  1050
client Id:  1051
client Id:  1052
client Id:  1053
client Id:  1054
client Id:  1055
client Id:  1056
client Id:  1057
client Id:  1058
client Id:  1059
client Id:  1060
client Id:  1061
client Id:  1062
client Id:  1063
client Id:  1064
client Id:  1065
client Id:  1066
client Id:  1067
client Id:  1068
client Id:  1069
client Id:  1070
client Id:  1071
client Id:  1072
client Id:  1073
client Id:  1074
client Id:  1075
client Id:  1076
client Id:  1077
client Id:  1078
client Id:  1079
client Id:  1080
client Id:  1081
client Id:  1082
client Id:  1083
client Id:  1084
client Id:  1085
client Id:  1086
client Id:  1087
client Id:  1088
client Id:  10

client Id:  1513
client Id:  1514
client Id:  1515
client Id:  1516
client Id:  1517
client Id:  1518
client Id:  1519
client Id:  1520
client Id:  1521
client Id:  1522
client Id:  1523
client Id:  1524
client Id:  1525
client Id:  1526
client Id:  1527
client Id:  1528
client Id:  1529
client Id:  1530
client Id:  1531
client Id:  1532
client Id:  1533
client Id:  1534
client Id:  1535
client Id:  1536
client Id:  1537
client Id:  1538
client Id:  1539
client Id:  1540
client Id:  1541
client Id:  1542
client Id:  1543
client Id:  1544
client Id:  1545
client Id:  1546
client Id:  1547
client Id:  1548
client Id:  1549
client Id:  1550
client Id:  1551
client Id:  1552
client Id:  1553
client Id:  1554
client Id:  1555
client Id:  1556
client Id:  1557
client Id:  1558
client Id:  1559
client Id:  1560
client Id:  1561
client Id:  1562
client Id:  1563
client Id:  1564
client Id:  1565
client Id:  1566
client Id:  1567
client Id:  1568
client Id:  1569
client Id:  1570
client Id:  15

client Id:  1995
client Id:  1996
client Id:  1997
client Id:  1998
client Id:  1999
client Id:  2000
client Id:  2001
client Id:  2002
client Id:  2003
client Id:  2004
client Id:  2005
client Id:  2006
client Id:  2007
client Id:  2008
client Id:  2009
client Id:  2010
client Id:  2011
client Id:  2012
client Id:  2013
client Id:  2014
client Id:  2015
client Id:  2016
client Id:  2017
client Id:  2018
client Id:  2019
client Id:  2020
client Id:  2021
client Id:  2022
client Id:  2023
client Id:  2024
client Id:  2025
client Id:  2026
client Id:  2027
client Id:  2028
client Id:  2029
client Id:  2030
client Id:  2031
client Id:  2032
client Id:  2033
client Id:  2034
client Id:  2035
client Id:  2036
client Id:  2037
client Id:  2038
client Id:  2039
client Id:  2040
client Id:  2041
client Id:  2042
client Id:  2043
client Id:  2044
client Id:  2045
client Id:  2046
client Id:  2047
client Id:  2048
client Id:  2049
client Id:  2050
client Id:  2051
client Id:  2052
client Id:  20

client Id:  2477
client Id:  2478
client Id:  2479
client Id:  2480
client Id:  2481
client Id:  2482
client Id:  2483
client Id:  2484
client Id:  2485
client Id:  2486
client Id:  2487
client Id:  2488
client Id:  2489
client Id:  2490
client Id:  2491
client Id:  2492
client Id:  2493
client Id:  2494
client Id:  2495
client Id:  2496
client Id:  2497
client Id:  2498
client Id:  2499
client Id:  2500
client Id:  2501
client Id:  2502
client Id:  2503
client Id:  2504
client Id:  2505
client Id:  2506
client Id:  2507
client Id:  2508
client Id:  2509
client Id:  2510
client Id:  2511
client Id:  2512
client Id:  2513
client Id:  2514
client Id:  2515
client Id:  2516
client Id:  2517
client Id:  2518
client Id:  2519
client Id:  2520
client Id:  2521
client Id:  2522
client Id:  2523
client Id:  2524
client Id:  2525
client Id:  2526
client Id:  2527
client Id:  2528
client Id:  2529
client Id:  2530
client Id:  2531
client Id:  2532
client Id:  2533
client Id:  2534
client Id:  25

client Id:  2959
client Id:  2960
client Id:  2961
client Id:  2962
client Id:  2963
client Id:  2964
client Id:  2965
client Id:  2966
client Id:  2967
client Id:  2968
client Id:  2969
client Id:  2970
client Id:  2971
client Id:  2972
client Id:  2973
client Id:  2974
client Id:  2975
client Id:  2976
client Id:  2977
client Id:  2978
client Id:  2979
client Id:  2980
client Id:  2981
client Id:  2982
client Id:  2983
client Id:  2984
client Id:  2985
client Id:  2986
client Id:  2987
client Id:  2988
client Id:  2989
client Id:  2990
client Id:  2991
client Id:  2992
client Id:  2993
client Id:  2994
client Id:  2995
client Id:  2996
client Id:  2997
client Id:  2998
client Id:  2999


In [27]:
file_name = f"Testset_{num_client}.pkl"

In [28]:
data=[test_data, test_label]

In [29]:
save_file(file_name,data)